In [8]:
import numpy as np
import random

from qat.model.builder import QuantumHardwareModelBuilder
from qat.model.component import make_refdict
from qat.model.device import PhysicalBaseband, PhysicalChannel, Qubit, Resonator, PulseChannel
from qat.model.hardware_model import QuantumHardwareModel

from qat.purr.compiler.devices import ChannelType

In [9]:
def build_hardware(n_qubits=4, connectivity=None):

    builder = QuantumHardwareModelBuilder()
    
    for q_index in range(n_qubits):
        
        baseband_r = builder.add_physical_baseband(
                            frequency=np.random.uniform(1e05, 1e07), 
                            if_frequency=np.random.uniform(1e05, 1e07)
                        )
        
        baseband_q = builder.add_physical_baseband(
                            frequency=np.random.uniform(1e05, 1e07), 
                            if_frequency=np.random.uniform(1e05, 1e07)
                        )

        physical_channel_r = builder.add_physical_channel(
                            baseband=baseband_r,
                            sample_time=np.random.uniform(1e-10, 1e-08), 
                            phase_iq_offset=np.random.uniform(0.0, 1.0),
                            bias=np.random.uniform(-1.0, 1.0),
                            acquire_allowed=True)
        
        physical_channel_q = builder.add_physical_channel(
                            baseband=baseband_q,
                            sample_time=np.random.uniform(1e-10, 1e-08), 
                            phase_iq_offset=np.random.uniform(0.0, 1.0),
                            bias=np.random.uniform(-1.0, 1.0),
                            acquire_allowed=False)

        resonator = builder.add_resonator(
                                frequency=np.random.uniform(1e06, 200e06),
                                physical_channel=physical_channel_r)
        
        qubit = builder.add_qubit(index=q_index, 
                                  frequency=np.random.uniform(1e08, 5e09),
                                  physical_channel=physical_channel_q,
                                  measure_device=resonator)
    
    builder.add_connectivity(connectivity=connectivity)
        
    return builder.model



In [10]:
n_qubits = 10

qubits = list(range(0, n_qubits))
random.shuffle(qubits)
connectivity = [(q1, q2) for q1, q2 in zip(qubits[: len(qubits) // 2], qubits[len(qubits) // 2 :])]

hw = build_hardware(n_qubits=n_qubits, connectivity=connectivity)

hw2 = QuantumHardwareModel(**hw.model_dump())

hw._deepequals(hw2)

True